# Dealing with BIG data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as sp_la

One more day with Boston housing data.

In [ ]:
data = np.array(np.genfromtxt('data/boston.csv', delimiter=',', skip_header=2, dtype=float, encoding='utf-8'))  
split = np.array_split(data, 10)
train = np.vstack((split[0], split[1], split[3], split[4], split[5], split[6], split[7], split[8], split[9]))
test = split[2]

In [ ]:
databig = np.vstack([data]*1000)
print(databig.shape)
print(data.shape)

(From day 15, which in turn was from day 12)

In [ ]:
def getSummaryStatistics(data):
    return np.array([data.max(axis=0), data.min(axis=0), data.mean(axis=0, dtype=int)])

def getShapeType(data):
    return (data.shape, data.dtype)

def fitlstsq(data, independent, dependent):
    # These are our independent variable(s)
    x = data[np.ix_(np.arange(data.shape[0]), independent)]
    # We add a column of 1s for the intercept
    A = np.hstack((np.array([np.ones(x.shape[0])]).T, x))
    # This is the dependent variable 
    y = data[:, dependent]
    # This is the regression coefficients that were fit, plus some other results
    c, res, _, _ = sp_la.lstsq(A, y)
    return c

def fitnorm(data, independent, dependent):
    # These are our independent variable(s)
    x = data[np.ix_(np.arange(data.shape[0]), independent)]
    # We add a column of 1s for the intercept
    A = np.hstack((np.array([np.ones(x.shape[0])]).T, x))
    # This is the dependent variable 
    y = data[:, dependent]
    # This is the regression coefficients that were fit, plus some other results
    c = sp_la.inv(A.T.dot(A)).dot(A.T).dot(y)
    return c

def fitqr(data, independent, dependent):
    # These are our independent variable(s)
    x = data[np.ix_(np.arange(data.shape[0]), independent)]
    # We add a column of 1s for the intercept
    A = np.hstack((np.array([np.ones(x.shape[0])]).T, x))
    # This is the dependent variable 
    y = data[:, dependent]
    # This is the regression coefficients that were fit, plus some other results
    Q, R = sp_la.qr(A)
    print(A.shape)
    print(Q.shape)
    print(R.shape)
    c = sp_la.solve_triangular(R, Q.T.dot(y))
    return c

def predict(data, independent, c):
    # These are our independent variable(s)
    x = data[np.ix_(np.arange(data.shape[0]), independent)]
    # We add a column of 1s for the intercept
    A = np.hstack((np.array([np.ones(x.shape[0])]).T, x))
    return np.dot(A, c)

def plotxyyhat(x, y, c):
    plt.plot(x, y, 'o', label='data')
    xCurve = np.linspace(np.min(x), np.max(x))
    yCurve = c[0]
    for i in range(1, len(c)):
        yCurve += c[i]*(xCurve**i)
    plt.plot(xCurve, yCurve, label='least squares fit, linear')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend(framealpha=1, shadow=True)
    plt.grid(alpha=0.25)
    plt.show()

In [ ]:
%%time
fitlstsq(data, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 12)

In [ ]:
%%time
fitnorm(databig, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 12)

In [ ]:
%%time
fitqr(databig, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 12)

## Gradient Descent

Let's talk about one more way to calculate a  regression. We already know three *analytical* methods. What are they?

This method is an *approximation* method. So it's not guaranteed to give us a correct answer. But it may be handy when the data is so gnarly that the analytical methods fail, or when the data is large (> 10000 data points, > 1000 variables). 

For this discussion I'm going to refer to linear regression with one independent variable, but the method is general.

All the back on day 12 we defined the MSSE as the *loss function* we wanted to *minimize* for regression. 
$$MSSE = 1 / N \sum_{i=1}^N (r_i)^2 = 1/N \sum_{i=1}^N (y_i - \hat{y}_i)^2$$
This method is an iterative step by step method. We take a step, figure out if we are going towards the *minimum loss* and if so we take another step. 

* At first we pick random values for $m$ and $b$ (our slope and intercept).
* At each step we:
  1. calculate the partial derivative of $MSSE$ with respect to $m$ and to $b$, and plug in $x$, $y$, $m$ and $b$.
    * partial derivative of $MSSE$ ($p$) with respect to $m$: $$\frac{\partial p}{\partial m} = 1/N \sum_{i=1}^N \frac{\partial }{\partial m} (y_i - \hat{y}_i)^2 = 1/N \sum_{i=1}^N \frac{\partial }{\partial m} (y_i - (m x_i + b))^2 = 1 / N \sum_{i=1}^N -2 x_i (y_i - (m x_i + b)) = -2 / N \sum_{i=1}^N x_i (y_i - \hat{y}_i)$$
    * partial derivative of $MSSE$ ($p$) with respect to $b$: $$\frac{\partial p}{\partial b} = 1/N \sum_{i=1}^N \frac{\partial }{\partial b} (y_i - \hat{y}_i)^2 = 1/N \sum_{i=1}^N \frac{\partial }{\partial b} (y_i - (m x_i + b))^2 = 1 / N \sum_{i=1}^N -2 (y_i - (m x_i + b)) = -2 / N \sum_{i=1}^N (y_i - \hat{y}_i)$$
  2. update $m$ and $b$ as follows:
    * $m = m - lr * \frac{\partial p}{\partial m}$
    * $b = b - lr * \frac{\partial p}{\partial b}$
    where $lr$ is a *learning rate* set, hopefully, big enough that we don't have to step forever, and small enough that we don't overshoot our goal (the minimum loss)
* We stop after a preset number of steps (*epochs*) or after the change in loss stops getting smaller

In [ ]:
from numpy.random import default_rng

def gradient_descent(data, independent, dependent, lr, epochs):
    # initialize m and b
    rng = default_rng()
    c = rng.standard_normal(2)
    # set n, x and y for readability of the method
    n = data.shape[0]
    x = data[np.ix_(np.arange(data.shape[0]), independent)]
    y = data[:, dependent]
    A = np.hstack((np.array([np.ones(x.shape[0])]).T, x))
    for i in range(epochs):
        yhat = np.dot(A, c)
        # how are we doing on MSSE?
        print((1/n) * np.sum(y - yhat)**2)
        if ((1/n) * np.sum(y - yhat)**2) < 0.00001:
            return c
        # fill in the partial derivatives
        dpdm = (-2/n) * np.sum(x * (y - yhat))
        dpdb = (-2/n) * np.sum(y - yhat)
        # update c
        c = c - np.array([lr * dpdm, lr * dpdb])
#        plotxyyhat(x, data[:, dependent], c)
    return c

In [ ]:
def rsquared(y, yhat):
    if len(y) != len(yhat):
        print("Need y and yhat to be the same length!")
        return 0
    return 1 - (((y - yhat)**2).sum() / ((y - y.mean())**2).sum())

capprox = gradient_descent(data, [7], 12, 0.0001, 1000)
cexact = fit(data, [7], 12)

# And how does it look on the test data that we held out?
print(capprox, cexact)
print(rsquared(test[:, 12], predict(test, [7], capprox)))
print(rsquared(test[:, 12], predict(test, [7], cexact)))

# Homework next week: boston x 50, lstsq, normal, qr, gd

## Stepwise Regression

Let's talk about dealing with data that has many variables (features). How do we know which ones to *use*?

One way to go about this is stepwise regression.

For stepwise regression we use a modification of $R^2$, $${R^2}_{adj} = 1 - \frac{(1-R^2)(N-1)}{N-k-1}$$
where $N$ is the number of variables, and $k$ is the number of variables in $A$.

Stepwise regression works like this:

1. Initialize $A$ to be just the leading column of 1s (because we know we will have an intercept).

2. Then while the improvements in ${R^2}_{adj}$ are > 0 and there remain independent variables not yet added:
  * calculate a regression using $A$ and each variable not yet in $A$, and 
  * add the one with the highest ${R^2}_{adj}$ to $A$.

Let's try this on the Boston housing data!

For this exercise, I'm going to use linear regression only.


We define our adjusted $R^2$.

In [ ]:
def rsquared(y, yhat):
    if len(y) != len(yhat):
        print("Need y and yhat to be the same length!")
        return 0
    return 1 - (((y - yhat)**2).sum() / ((y - y.mean())**2).sum())

def rsquared_adj(y, yhat, n, k):
    return 1 - (((1-rsquared(y, yhat))*(n-1)) / (n-k-1))

We define stepwisefit.

In [ ]:
def stepwisefit(data, dependent):
    # I'm just going to store the indices of the independent variables we are keeping
    colsinA  = []
    while(len(colsinA) < data.shape[1]):
        print(colsinA)
        best_independent = -1
        # extension: set best_rsa to be the rsa of A 
        best_rsa = 0.0
        for independent in range(data.shape[1]):
            if independent != dependent and independent not in colsinA:
                c = fit(data, colsinA + [independent], dependent)
                yhat = predict(data, colsinA + [independent], c)
                rsa = rsquared_adj(data[:, dependent], yhat, data.shape[1], len(colsinA))
                print(rsa, independent, best_rsa, best_independent)
                if rsa > best_rsa:
                    best_rsa = rsa
                    best_independent = independent
        if rsa <= 0:
            return colsinA
        if best_independent > -1:
            colsinA.append(best_independent)
    return colsinA

We try it on the Boston data.

In [ ]:
colsinA = stepwisefit(data, 12)
print(colsinA)

# And how does it look on the test data that we held out?

This is not the only way to do stepwise regression: we could start with all the variables, and then iteratively remove one; or we could add *and* remove in each step (see https://en.wikipedia.org/wiki/Stepwise_regression).

This is also not the only way to figure out whether variables should be included in a regression. You might have knowledge ahead of time. If you don't:
* You might calculate correlations
* You might look at a pair plot

## The curse of dimensionality

Of course when data is very multidimensional, everything becomes slower and more complicated:
* How can you look at your data? There's too much there!
* How can you normalize your data? Without being able to look at it, what normalizations would you know to apply?
* How can you fit a regression to your data? Which variables would you choose?

In fact, data in a high dimensional space just is different:
* there are more extreme values. In a 2 dimensional unit square, the probability that a random point is within 0.001 of the border is 0.004, but in a 10000 dimensional unit hypercube, the probability is > 0.999.
* the distances between points are bigger. In a 2 dimensional unit square, the average distance between two points is about 0.5. In a 3 dimensional unit cube, it's 0.66. But in a 10000 dimensional unit hypercube, it's about 408.25.

Here's the thing though: generally, only a small fraction of variables in a data set suffice to model the whole. So if we can identify them, we can *project* our data to a much smaller dimensional space and analyze the data in that space instead. The technique we will use for that is *principal component analysis* (PCA).